In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score


In [ ]:
# Loading dataset
(X_tr, y_tr), (X_te, y_te) = tf.keras.datasets.cifar10.load_data()


In [ ]:
# Filter out cats,dogs based on classes
train_mask = (y_tr == 3) | (y_tr == 5)
X_tr, y_tr = X_tr[train_mask.squeeze()], y_tr[train_mask.squeeze()]


In [ ]:
test_mask = (y_te == 3) | (y_te == 5)
X_te, y_te = X_te[test_mask.squeeze()], y_te[test_mask.squeeze()]


In [ ]:
#  labeling 0 for cats, 1 for dogs
y_tr = (y_tr == 5).astype(int)
y_te = (y_te == 5).astype(int)

In [ ]:
# Normalizing the  data
X_tr = X_tr.astype("float32") / 255
X_te = X_te.astype("float32") / 255


In [ ]:
#  scratch CNN model 
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu', name='feature_extraction'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])


In [ ]:
# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Training the model
model.fit(X_tr, y_tr, epochs=20, validation_data=(X_te, y_te))


Epoch 1/20
313/313 [==============================] - 36s 93ms/step - loss: 0.6703 - accuracy: 0.5765 - val_loss: 0.6258 - val_accuracy: 0.6370
Epoch 2/20
313/313 [==============================] - 19s 62ms/step - loss: 0.5935 - accuracy: 0.6819 - val_loss: 0.5601 - val_accuracy: 0.7015
Epoch 3/20
313/313 [==============================] - 20s 63ms/step - loss: 0.5372 - accuracy: 0.7273 - val_loss: 0.5250 - val_accuracy: 0.7370
Epoch 4/20
313/313 [==============================] - 22s 72ms/step - loss: 0.4915 - accuracy: 0.7591 - val_loss: 0.5160 - val_accuracy: 0.7495
Epoch 5/20
313/313 [==============================] - 20s 63ms/step - loss: 0.4484 - accuracy: 0.7832 - val_loss: 0.4901 - val_accuracy: 0.7645
Epoch 6/20
313/313 [==============================] - 20s 64ms/step - loss: 0.4036 - accuracy: 0.8147 - val_loss: 0.5031 - val_accuracy: 0.7600
Epoch 7/20
313/313 [==============================] - 22s 71ms/step - loss: 0.3590 - accuracy: 0.8391 - val_loss: 0.4950 - val_accuracy:

In [ ]:
# Extracting  features from fully connected layers
feature_extractor = Model(inputs=model.input, outputs=model.get_layer('feature_extraction').output)
X_tr_feat = feature_extractor.predict(X_tr)
X_te_feat = feature_extractor.predict(X_te)


63/63 [==============================] - 1s 15ms/step


In [ ]:
# Evaluation of the model CNN
y_pred_cnn = (model.predict(X_te) > 0.5).astype("int32")
accuracy_cnn = accuracy_score(y_te, y_pred_cnn)
f1_cnn = f1_score(y_te, y_pred_cnn)
precision_cnn = precision_score(y_te, y_pred_cnn)
recall_cnn = recall_score(y_te, y_pred_cnn)

63/63 [==============================] - 1s 16ms/step


In [ ]:
print("CNN:")
print("Accuracy: ", accuracy_cnn)
print("F1 Score: ", f1_cnn)
print("Precision: ", precision_cnn)
print("Recall: ", recall_cnn)

CNN:
Accuracy:  0.757
F1 Score:  0.7460815047021943
Precision:  0.7811816192560175
Recall:  0.714


In [ ]:
# Training SVM with the features obtained
svm = SVC(kernel='linear')
svm.fit(X_tr_feat, y_tr.ravel())


SVC(kernel='linear')

In [ ]:
# Evaluating SVM with test data
y_pred = svm.predict(X_te_feat)
accuracy = accuracy_score(y_te, y_pred)
f1 = f1_score(y_te, y_pred)
precision = precision_score(y_te, y_pred)
recall = recall_score(y_te, y_pred)


In [ ]:
print(f"SVM Accuracy: {accuracy:.2f}")
print(f"SVM F1 Score: {f1:.2f}")
print(f"SVM Precision: {precision:.2f}")
print(f"SVM Recall: {recall:.2f}")

SVM Accuracy: 0.76
SVM F1 Score: 0.76
SVM Precision: 0.76
SVM Recall: 0.76
